## Meta matching v1.0
This jupyter notebook demonstrates you how to load and use meta-matching algorthm. In this demonstration, we performed Meta-matching (DNN) stacking with 100 example subjects.

Package needed (and version this jupyter notebook tested):
* Numpy (1.16.4)
* Scipy (1.0.0)
* PyTorch (1.3.1)
* Scikit-learn (0.21.3)

### Step 0. Setup
Please modify the `path_repo` below to your repo position:

In [1]:
path_repo = "../" #'/home/the/deepGround/code/2002/Meta_matching_models/'

In [2]:
# initialization and random seed set

import os
import sys
import random
import scipy
import torch
import sklearn
import numpy as np

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

import warnings
warnings.filterwarnings("ignore")

Please modify the gpu number here if you want to use different gpu. If the gpu you assigned not availiable, it will assign to cpu

In [3]:
gpu = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

### Step 1. load data
Load the example data that we provided, it contains 
* Example input functional connectivity (FC) `x` with size of (100, 87571)
    * 100 is number of subjects
    * 87571 is flatten vector of 419 by 419 FC (419*418/2=87571)
* Example output phenotypes `y` with size of (100, 3)
    * 3 is number of phenotypes.

In [4]:
path_v1 = os.path.join(path_repo, 'v1.0')
path_data = os.path.join(path_repo, 'data')
npz = os.path.join(path_data, 'meta_matching_example_data.npz')
npz = np.load(npz)
print(npz.files)
x_input = npz['x']
y_input = npz['y']
print(x_input.shape, y_input.shape)

['x', 'y']
(100, 87571) (100, 3)


### Step 2. Split data
Here, we also split 100 subjects to 80/20, where 80 for training, and 20 for test.

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_input, y_input, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(80, 87571) (20, 87571) (80, 3) (20, 3)


### Step 3. Prepare data for PyTorch model
Then we prepare data for DNN model, we will input both the `x_train` and `x_test` into the model to get the predicted phenotypes. 

For meta-matching (DNN) stacking, we do not need real phenotype for the DNN model, I created all zeros `y_dummy` just for function requirement. In some other cases, like meta-matching (DNN) finetuning, you need to use real phenotype data here.

In [6]:
from torch.utils.data import DataLoader
sys.path.append(os.path.join(path_repo, "utils"))
from CBIG_model_pytorch import multi_task_dataset

batch_size = 16
y_dummy = np.zeros(y_train.shape)
dset_train = multi_task_dataset(x_train, y_dummy, True)
trainLoader = DataLoader(dset_train,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=1)

y_dummy = np.zeros(y_test.shape)
dset_test = multi_task_dataset(x_test, y_dummy, True)
testLoader = DataLoader(dset_test,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=1)

### Step 4. load model
Here we load the meta-matching model saved, it is a DNN that takes FC as input and output 67 phenotypes prediction trained on 67 UK Biobank phenotypes

In [7]:
path_model_weight = os.path.join(path_v1, 'meta_matching_v1.0_model.pkl_torch')  
net = torch.load(path_model_weight, map_location=device)
net.train(False)
print(net)

dnn_4l(
  (fc1): Sequential(
    (0): Dropout(p=0.28836)
    (1): Linear(in_features=87571, out_features=118, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(118, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc2): Sequential(
    (0): Dropout(p=0.28836)
    (1): Linear(in_features=118, out_features=445, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(445, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc3): Sequential(
    (0): Dropout(p=0.28836)
    (1): Linear(in_features=445, out_features=353, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(353, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc4): Sequential(
    (0): Dropout(p=0.28836)
    (1): Linear(in_features=353, out_features=67, bias=True)
  )
)


### Step 5. DNN model predict
Here we apply the DNN trained on 67 UK Biobank phenotypes to predict the 67 phenotypes on `x_train` and `x_test`. We will get the predicted 67 phenotypes on both 80 training subjects and 20 test subjects.

In [8]:
y_train_pred = np.zeros((0, 67))
for (x, _) in trainLoader:
    x= x.to(device)
    outputs = net(x)
    y_train_pred = np.concatenate((y_train_pred, outputs.data.cpu().numpy()), axis=0)
print(y_train_pred.shape, '\n', y_train_pred)

y_test_pred = np.zeros((0, 67))
for (x, _) in testLoader:
    x= x.to(device)
    outputs = net(x)
    y_test_pred = np.concatenate((y_test_pred, outputs.data.cpu().numpy()), axis=0)
print(y_test_pred.shape, '\n', y_test_pred)

(80, 67) 
 [[-0.17227206  0.00659013 -0.30208844 ... -0.15928993 -0.38077328
   0.03831927]
 [ 0.08401433 -0.20126955  0.62567234 ...  0.14059746 -0.00804296
   0.46316051]
 [ 0.13499442  0.1402549  -0.51653892 ...  0.01679243 -0.21242383
  -0.15089571]
 ...
 [ 0.14017355 -0.13507904  0.06178962 ...  0.14889181  0.00375715
   0.00102939]
 [-0.00132075 -0.20964415  0.35207596 ...  0.17323519 -0.12568414
  -0.07328446]
 [-0.14659929  0.09759449 -0.2422424  ... -0.07367411 -0.56735659
  -0.06110723]]
(20, 67) 
 [[-0.0709152  -0.20356582  0.30995744 ...  0.17636669 -0.36347187
  -0.13356218]
 [-0.01349916 -0.13479878  0.26539761 ...  0.22685154  0.03282836
  -0.25569448]
 [ 0.03873895 -0.22324213  0.34538239 ...  0.21638942 -0.10278907
  -0.06968378]
 ...
 [ 0.01483438 -0.2722261   0.38793978 ...  0.13031584 -0.24093716
   0.35386074]
 [-0.11500868 -0.14910071  0.18525353 ...  0.11365268 -0.32046545
  -0.13270389]
 [ 0.13167745  0.13831379 -0.34880364 ...  0.00669172  0.02045992
  -0.12482

### Step 6. Stacking
Perform stacking with `y_train_pred`, `y_test_pred`, `y_train`, where we use the prediction of 80 subjects `y_train_pred` (input) and real data `y_train` (output) to train the stacking model (you can either use all 67 source phenotypes for stacking, or select top K source phenotypes relevant to the target phenotype, like we mentioned in our paper; it turns out that these 2 ways achieves similar performances), then we applied the model to `y_test_pred` to get final prediction of 3 phenotypes on 20 subjects.

#### Hyperparameter Tuning 
In `stacking()` function, we set the range of `alpha` as `[0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20]`. You are weclomed to modify the range of `alpha` to get better performance on your own data.

In [9]:
from CBIG_model_pytorch import stacking
y_test_final=np.zeros((y_test_pred.shape[0], y_train.shape[1]))
for i in range(y_train.shape[1]):
    # For each test phenotype, perform stacking by developing a KRR model
    y_test_temp, _ = stacking(y_train_pred, y_test_pred, y_train[:,i].view(), [0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20])
    y_test_final[:,i] = y_test_temp.flatten()
print(y_test_final.shape, '\n', y_test_final)

(20, 3) 
 [[57.24721265 25.93593734 27.4226106 ]
 [57.17467899 22.06165651 27.09655359]
 [57.36884547 24.58059507 23.47093781]
 [37.18787426 11.75935299 17.75585082]
 [49.15730921 18.46283718 20.20718313]
 [47.47035798 20.04291131 19.9608776 ]
 [54.40567731 24.73385541 23.27536312]
 [40.99368241 15.31938305 21.09597124]
 [39.04512584 13.11091205 24.61155908]
 [55.31271847 23.59101645 23.6640419 ]
 [53.26186813 21.25681156 32.77595402]
 [69.68916503 30.63487344 33.29932061]
 [57.15708736 22.6005652  28.75085869]
 [63.07878367 27.04353219 36.24263957]
 [61.76455787 25.88692146 30.91435613]
 [45.64961507 18.33914249 26.59452772]
 [32.08764725 10.42886165 27.66786859]
 [52.84634842 19.99914532 21.17068412]
 [56.24633603 23.80775    29.41020396]
 [30.1881153  11.68766442 18.1356171 ]]


### Step 7. Evaluation
Evaluate the prediction performance.

In [10]:
from scipy.stats.stats import pearsonr
corr = np.zeros((y_train.shape[1]))
for i in range(y_train.shape[1]):
    corr[i] = pearsonr(y_test_final[:, i], y_test[:, i])[0]
print(corr)

[0.24716778 0.37310409 0.22338988]


### Step 8. Haufe transform predictive network features (PNFs) computation
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each element of FC on the 80 training subjects. The final PNF is in shape of (87571, 3), where 87571 is number of 419 by 419 FC elements, and 3 is number of phenotypes.

In [11]:
from CBIG_model_pytorch import covariance_rowwise

y_train_haufe, _ = stacking(y_train_pred, y_train_pred, y_train)
print(y_train_haufe.shape)
cov = covariance_rowwise(x_train, y_train_haufe)
print(cov, '\n', cov.shape)

(80, 3)
[[ 0.18386811  0.11068117  0.01352517]
 [ 0.1119323   0.07715998 -0.04188844]
 [ 0.14536112  0.09807391 -0.04420557]
 ...
 [ 0.21679985  0.09523134  0.04394229]
 [ 0.19566549  0.09021557  0.03804702]
 [ 0.24579299  0.12446289 -0.00408745]] 
 (87571, 3)


### Step 9. Haufe transform predictive network features (PNFs) computation for training phenotypes
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each training phenotypes on the 80 training subjects. The final PNF is in shape of (67, 3), where 67 is number of training phenotypes, and 3 is number of phenotypes.

In [12]:
from CBIG_model_pytorch import covariance_rowwise

cov = covariance_rowwise(y_train_pred, y_train_haufe)
print(cov, '\n', cov.shape)

[[ 1.09301723e-01  3.93908557e-02  1.50070203e-03]
 [-3.24056429e-01 -1.63686217e-01  1.19563060e-02]
 [ 8.72238400e-01  3.97970535e-01  2.16038049e-01]
 [ 1.64926498e-01  8.13263564e-02  6.21016424e-02]
 [ 5.25558768e-02 -1.65186988e-02 -1.95600160e-01]
 [-2.92966903e-01 -2.00420830e-01 -2.87878264e-01]
 [ 9.16181168e-02  4.24523333e-02  1.85445675e-01]
 [ 1.52624282e-01  8.78352365e-02  2.46047657e-01]
 [ 7.58827271e-01  3.20920724e-01  1.40667180e-01]
 [-2.34888579e-02 -6.77215721e-02 -1.80829813e-01]
 [ 3.62779024e-01  1.59681171e-01  6.86977733e-02]
 [-4.80388511e-01 -2.90244528e-01 -1.80062811e-01]
 [-2.35075035e-01 -1.05488600e-01 -1.44898591e-01]
 [ 7.05744314e-01  3.93537910e-01  7.06486033e-01]
 [ 3.05856512e-01  1.54111843e-01  2.50461209e-01]
 [ 2.27410649e+00  1.12351113e+00  6.99330271e-01]
 [-7.61003883e-01 -4.18166826e-01 -3.64184371e-01]
 [-3.24148143e-01 -8.51299384e-02 -1.40108139e-01]
 [-7.59052647e-01 -3.86190192e-01 -5.27179275e-01]
 [-2.87858458e-02 -6.68951008e-